In [1]:
pwd

'/home/jovyan/private/PycharmProjects/FlexGen-EHR-master/FlexGen-master'

In [2]:
from models.cvae import VariationalAutoencoder, vae_loss_fn
from models.ddpm import DDPM, ContextUnet
from models.flexgen import flexgen
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import json
import pandas as pd
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

class MIMICDATASET(Dataset):
    def __init__(self, x_t,x_s, ys, yt, train=None, transform=None):
        # Transform
        self.transform = transform
        self.train = train
        self.xt = x_t
        self.xs = x_s
        self.ys = ys
        self.yt = yt
        m = x_s.shape[0]
        num_miss = 500
        self.miss_t = np.random.permutation(np.arange(m))[0: num_miss]
        self.miss_s = np.random.permutation(np.arange(m))[0: num_miss]

    def return_data(self):
        return self.xt, self.xs, self.ys, self.yt

    def __len__(self):
        return len(self.xt)


    def __getitem__(self, idx):
        # sample 처리
        if idx in self.miss_t:
            # sample과 동일한 크기의 영 텐서 생성
            sample = torch.zeros_like(self.xt[0])
        else:
            sample = self.xt[idx]

        # stat 처리
        if idx in self.miss_s:
            # stat과 동일한 크기의 영 텐서 생성
            stat = torch.zeros_like(self.xs[0])
        else:
            stat = self.xs[idx]

        sample_ys = self.ys[idx]
        sample_yt = self.yt[idx]

        # 레이블이 텐서인지 확인하고, 아니면 텐서로 변환
        if not isinstance(sample_ys, torch.Tensor):
            sample_ys = torch.tensor(sample_ys, dtype=torch.float32)
        if not isinstance(sample_yt, torch.Tensor):
            sample_yt = torch.tensor(sample_yt, dtype=torch.float32)

        return sample, stat, sample_ys, sample_yt

In [3]:
batch_size =  512
device = torch.device("cuda")
tasks = [
    'mortality_48h',
    'ARF_4h', 
    'ARF_12h',
    'Shock_4h',
    'Shock_12h',
]
task = tasks[0]
s = np.load('FIDDLE_mimic3/features/{}/s.npz'.format(task))
X = np.load('FIDDLE_mimic3/features/{}/X.npz'.format(task))
s_feature_names = json.load(open('FIDDLE_mimic3/features/{}/s.feature_names.json'.format(task), 'r'))
X_feature_names = json.load(open('FIDDLE_mimic3/features/{}/X.feature_names.json'.format(task), 'r'))
df_pop = pd.read_csv('FIDDLE_mimic3/population/{}.csv'.format(task))
x_s = torch.sparse_coo_tensor(torch.tensor(s['coords']), torch.tensor(s['data'])).to_dense().to(torch.float32)
x_t = torch.sparse_coo_tensor(torch.tensor(X['coords']), torch.tensor(X['data'])).to_dense().to(torch.float32)
x_t = x_t.sum(dim=1).to(torch.float32)

idx_t = np.random.permutation(np.arange(x_t.shape[0]))[0: int(0.8 * x_t.shape[0])]
idx_s = np.random.permutation(np.arange(x_s.shape[0]))[0: int(0.8 * x_s.shape[0])]
x_s = x_s[idx_s]
x_t = x_t[idx_t]
y = torch.tensor(df_pop["mortality_LABEL"].values).to(torch.float32)
ys = y[idx_s]
yt = y[idx_t]
dataset_train_object = MIMICDATASET(x_t, x_s, ys, yt, train=True, transform=False)
train_loader = DataLoader(dataset_train_object, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=False)

In [41]:
# 필요한 값만 언팩
tmp_samples, sta_samples, _, _ = next(iter(train_loader))
feature_dim_s = sta_samples.shape[1]
feature_dim_t = tmp_samples.shape[1]

svae = VariationalAutoencoder(feature_dim_s).to(device)
tvae = VariationalAutoencoder(feature_dim_t).to(device)
tvae = torch.load('saved_models/vae_tmp.pt')
svae = torch.load('saved_models/vae_stat.pt')
svae.eval()
tvae.eval()
n_epoch = 1
n_T = 50 
device = "cuda"
n_classes = 2
n_feat = 256  
lrate = 1e-4
save_model = True
w = 0.1
ddpm = DDPM(nn_model=ContextUnet(in_channels=1, n_feat=n_feat, n_classes=2), betas=(1e-4, 0.02), n_T=n_T, device=device, drop_prob=0.1)
ddpm.to(device)
trainer = flexgen(tvae, svae, ddpm,train_loader,epochs=n_epoch, model_path='Synthetic_MIMIC/diff.pt')
trainer.train_epoch()
trainer.generate(5000, 0)

/tmp/ipykernel_89210/1879940627.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tvae = torch.load('saved_models/vae_tmp.pt')
/tmp/ipykernel_89210/1879940627.py:9: Future

epoch 0


  0%|          | 0/14 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 3)